In [1]:
def readJson(f):
    for l in open(f):
        yield eval(l)

In [102]:
from collections import defaultdict
userdict = {}
itemdict = {}

for l in readJson('amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    if user not in userdict:
        userdict[user] = []
    userdict[user].append(item)
    if item not in itemdict:
        itemdict[item] = []
    itemdict[item].append(user)

In [103]:
import numpy as np
user_purchase_count = []
item_purchase_count = []
for user in userdict:
    user_purchase_count.append(len(userdict[user]))
for item in itemdict:
    item_purchase_count.append(len(itemdict[item]))
mean_user_purchase = np.mean(user_purchase_count)
mean_item_purchase = np.mean(item_purchase_count)

In [104]:
print ('on average a user buy %f items'%(mean_user_purchase))
print ('on average an item gets bought by %f users'%(mean_item_purchase))

on average a user buy 1.962023 items
on average an item gets bought by 5.841633 users


In [36]:
import re
meta = []
for l in readJson('amazon_reviews_Electronics/meta.json'):
    item,categories,price = l['asin'],l['categories'],l['price']
    meta.append({'item': item, 'categories': re.sub(r'\W+', ' ', categories), 'price': price})

In [40]:
import pandas as pd
meta_df = pd.DataFrame(meta).set_index('item')
meta_df.head()

,categories,price
item,,
ed2538e73bd60a71a3cdb5c1d7e1558fa04aa05c4856a529021ef3cc8e06cd64,Electronics Computers Accessories Cables Acce...,
805e2f92a47fa1949b03bb6eb623ba85c58fb7f252042d6c74cac1a395a7374f,Electronics Computers Accessories Cables Acce...,
24a01893dbd46c9c695b53d671fb2c461e131b1284f93238bb24d74c20cd3079,Electronics Computers Accessories PDAs Handhe...,8.15
a6ed402934e3c1138111dce09256538afb04c566edf37c16b9ba099d23afb764,Electronics Accessories Supplies Audio Video ...,23.36
8939d791e9dd035aa58da024ace69b20d651cea4adf6159d984872b44f663301,Electronics GPS Navigation Vehicle GPS Trucki...,299.99


In [88]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(meta_df.categories)
cat_vec = vectorizer.transform(meta_df.categories)

In [163]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

svd = TruncatedSVD(n_components=500, algorithm='arpack')
cat_lsa = svd.fit_transform(cat_vec)

cat_norm = normalize(cat_lsa, axis=0)
cat_df = pd.DataFrame(cat_norm, columns=list(range(500)),index=meta_df.index)
cat_df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
item,,,,,,,,,,,,,,,,,,,,,
ed2538e73bd60a71a3cdb5c1d7e1558fa04aa05c4856a529021ef3cc8e06cd64,0.002089,-0.001271,-0.000421,0.000344,-0.001200,0.000515,-0.000176,-0.001580,-0.000068,0.001695,...,0.000295,-0.000334,0.000172,-0.000348,-0.000544,-0.000284,-0.000034,0.000044,0.000425,0.000139
805e2f92a47fa1949b03bb6eb623ba85c58fb7f252042d6c74cac1a395a7374f,0.002089,-0.001271,-0.000421,0.000344,-0.001200,0.000515,-0.000176,-0.001580,-0.000068,0.001695,...,0.000295,-0.000334,0.000172,-0.000348,-0.000544,-0.000284,-0.000034,0.000044,0.000425,0.000139
24a01893dbd46c9c695b53d671fb2c461e131b1284f93238bb24d74c20cd3079,0.001506,-0.000649,0.000256,-0.000318,-0.000399,-0.000379,-0.000444,-0.001633,-0.000804,0.000866,...,0.000162,0.000069,-0.000029,-0.000044,-0.000023,-0.000002,-0.000001,-0.000080,0.000128,0.000043
a6ed402934e3c1138111dce09256538afb04c566edf37c16b9ba099d23afb764,0.001580,0.000304,-0.001679,0.000787,0.000617,-0.000664,-0.002765,0.001642,-0.002561,-0.000545,...,-0.000030,0.000002,-0.000052,-0.000107,0.000033,-0.000036,0.000020,-0.000095,0.000110,-0.000024
8939d791e9dd035aa58da024ace69b20d651cea4adf6159d984872b44f663301,0.000381,0.001177,0.000059,-0.001041,-0.000248,0.000175,-0.000180,-0.000392,0.000124,0.000725,...,0.000223,0.000291,-0.000187,-0.000296,-0.000176,-0.000089,0.000087,-0.000242,-0.000020,0.000370


In [180]:
price_df = pd.to_numeric(meta_df.price, errors='coerce')
price_norm = 500 * normalize(price_df.fillna(price_df.mean()))
price_norm
# meta_df1 = pd.concat([cat_df, meta_df.price], axis=1)
# meta_df1.head()

/Users/XiaoYixuan/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.35672565,  0.35672565,  0.04734516, ...,  0.10985239,
         0.14836753,  0.36197262]])

In [211]:
cat_df['price'] = price_norm.reshape(-1,1)
cat_df.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,price
item,,,,,,,,,,,,,,,,,,,,,
ed2538e73bd60a71a3cdb5c1d7e1558fa04aa05c4856a529021ef3cc8e06cd64,0.002089,-0.001271,-0.000421,0.000344,-0.001200,0.000515,-0.000176,-0.001580,-0.000068,0.001695,...,-0.000334,0.000172,-0.000348,-0.000544,-0.000284,-0.000034,0.000044,0.000425,0.000139,0.356726
805e2f92a47fa1949b03bb6eb623ba85c58fb7f252042d6c74cac1a395a7374f,0.002089,-0.001271,-0.000421,0.000344,-0.001200,0.000515,-0.000176,-0.001580,-0.000068,0.001695,...,-0.000334,0.000172,-0.000348,-0.000544,-0.000284,-0.000034,0.000044,0.000425,0.000139,0.356726
24a01893dbd46c9c695b53d671fb2c461e131b1284f93238bb24d74c20cd3079,0.001506,-0.000649,0.000256,-0.000318,-0.000399,-0.000379,-0.000444,-0.001633,-0.000804,0.000866,...,0.000069,-0.000029,-0.000044,-0.000023,-0.000002,-0.000001,-0.000080,0.000128,0.000043,0.047345
a6ed402934e3c1138111dce09256538afb04c566edf37c16b9ba099d23afb764,0.001580,0.000304,-0.001679,0.000787,0.000617,-0.000664,-0.002765,0.001642,-0.002561,-0.000545,...,0.000002,-0.000052,-0.000107,0.000033,-0.000036,0.000020,-0.000095,0.000110,-0.000024,0.135703
8939d791e9dd035aa58da024ace69b20d651cea4adf6159d984872b44f663301,0.000381,0.001177,0.000059,-0.001041,-0.000248,0.000175,-0.000180,-0.000392,0.000124,0.000725,...,0.000291,-0.000187,-0.000296,-0.000176,-0.000089,0.000087,-0.000242,-0.000020,0.000370,1.742708


In [212]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(5000, batch_size=5000, random_state=0)
kmeans.fit(cat_df)

MiniBatchKMeans(batch_size=5000, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=5000, n_init=3, random_state=0, reassignment_ratio=0.01,
        tol=0.0, verbose=0)

In [149]:
# count the number of users that buy same two items
com_user = {}
for user in userdict:
    for ii in userdict[user]:
        for ij in userdict[user]:
            if ii != ij:
                if str(ii)+' '+str(ij) in com_user:
                    com_user[str(ii)+' '+str(ij)] += 1
                else:
                    com_user[str(ii)+' '+str(ij)] = 1

In [113]:
# compute item jaccard similarity
item_sim = {}
alpha = 0.1
for pair in com_user:
#     print (pair)
    ii, ij = pair.split(' ')
    if ii not in item_sim:
        item_sim[ii] = {ij : 0}
    item_sim[ii][ij] = min(com_user[pair]/10, 1)*com_user[pair]/(len(itemdict[ii]) + len(itemdict[ij]) - com_user[pair] + alpha)

In [117]:
# min_sim = min([item_sim[x][y] for x in item_sim for y in item_sim[x]])
# scale = max([item_sim[x][y] for x in item_sim for y in item_sim[x]]) - min_sim
for x in item_sim:
    for y in item_sim[x]:
        item_sim[x][y] = (item_sim[x][y] - min_sim)/scale

In [224]:
itemCount = defaultdict(int)
totalPurchases = 0

for l in readJson('amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    itemCount[item] += 1
    totalPurchases += 1

mostPopular = [(itemCount[x], x) for x in itemCount]
mostPopular.sort(reverse=True)

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/2: break

In [271]:
predictions = open("amazon_reviews_Electronics/predictions_Purchase_1.txt", 'w')
# iternum = 0
for l in open("amazon_reviews_Electronics/pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
#     if iternum > 200:
#         break
#     iternum += 1

    user,item = l.strip().split('-')
        
    similarity = 0
    if user in userdict and item in item_sim:
        for other in item_sim[item]:
            if other in userdict[user]:
                similarity += item_sim[item][other]
        if len(userdict[user]) > 20*mean_user_purchase:
            similarity *= 2
        elif len(userdict[user]) > 10*mean_user_purchase:
            similarity *= 1.5
        elif len(userdict[user]) > 5*mean_user_purchase:
            similarity *= 1.2
        if len(itemdict[item]) > 20*mean_item_purchase:
            similarity *= 2
        elif len(itemdict[item]) > 10*mean_item_purchase:
            similarity *= 1.5
        elif len(itemdict[item]) > 5*mean_item_purchase:
            similarity *= 1.2
    flag1 = 1 if similarity > 0.3 else 0

    flag2 = 0
    count = 0
    if user in userdict:
        item_label = kmeans.labels_[np.where(cat_df.index==item)[0]][0]
        for tmp_item in userdict[user]:
            tmp_label = kmeans.labels_[np.where(cat_df.index==tmp_item)[0]][0]
            if item_label == tmp_label:
                count += 1
        if count/len(userdict[user]) > 0.2:
            flag2 = 1
    
    flag3 = 0
    if item in return1:
        flag3 = 1
    else:
        tmp_label = kmeans.labels_[np.where(cat_df.index==item)[0]][0]
        group  = cat_df.index[kmeans.labels_ == tmp_label]
        count_pop = 0
        for tmp_item in group:
            if tmp_item in return1:
                count_pop += 1
        if count_pop/len(group) > 0.5:
            flag3 = 1
    predict = 1 if flag1+flag2+flag3 > 0 else 0
#     print ('predict = %d flag1 = %d flag2 = %d flag3 = %d'%(predict,flag1,flag2,flag3))
    predictions.write(user + '-' + item + ',' + str(predict) + '\n')
    
predictions.close()